In [25]:
import psycopg2
import pandas as pd
pd.options.display.max_columns = 40

### Connect to the database

In [26]:
try:
    conn = psycopg2.connect("host=127.0.0.1 dbname=hockey_yt_project user=postgres password=password")
except psycopg2.Error as e:
    print("Error: Could not connect to database")
    print(e)

try:
    cur = conn.cursor()
except psycopg2.Error as e:
    print("Error: Could not get cursor to the database")
    print(e)

conn.set_session(autocommit=True)

#### create table for fact_scoring_log

In [27]:
try:
    cur.execute("CREATE TABLE IF NOT EXISTS fact_player_scoring_log (\
                id serial primary key, \
                tracked_player_name varchar,\
                point_number int,\
                team_year_gp_key varchar,\
                p varchar, \
                time varchar,\
                goal_strength varchar,\
                description varchar,\
                goal_scorer varchar,\
                primary_assist varchar,\
                secondary_assist varchar);")
    
except psycopg2.Error as e:
    print("Error: Could not create table")
    print(e)

In [28]:
gamelog_insert = ("""INSERT INTO fact_player_scoring_log(
                  tracked_player_name,
                  point_number,
                  team_year_gp_key,
                  p,
                  time,
                  goal_strength,
                  description,
                  goal_scorer,
                  primary_assist,
                  secondary_assist)
               VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
               """)

In [35]:
## load DF for dates
df_gamelog = pd.read_csv("/Users/mattrende/Documents/Coding/hockey-reference/YouTube Project/csv/out/fact_jack_2023_scoringlog.csv")
df_gamelog.head(2)

,tracked_player_name,Point_Number,Team_Game_Key,P,Time,Goal_Strength,Description,goal_scorer,primary_assist,secondary_assist
0,"Hughes, Jack",1,18-2023-R3,2,04:17,EV,Goal by Ondřej Palát assisted by Ryan Graves a...,"Palát, Ondřej","Graves, Ryan","Hughes, Jack"
1,"Hughes, Jack",2,18-2023-R3,3,12:49,EV,Goal by Dawson Mercer assisted by Yegor Sharan...,"Mercer, Dawson","Sharangovich, Yegor","Hughes, Jack"


In [36]:
try:
    for i, row in df_gamelog.iterrows():
        #print(list(row))
        cur.execute(gamelog_insert, list(row))

except psycopg2.Error as e:
    print("Error: could not insert rows")
    print(e)